# Notebook 07: Supervisor Prototype

Purpose:
- Implement a Supervisor layer for NEEL
- Validate whether AI reasoning should proceed
- Assign confidence levels and warnings
- Act as a safety and sanity gate before LLM reasoning

In [9]:
from dataclasses import dataclass
from typing import Dict, List

In [10]:
@dataclass
class SupervisorDecision:
    status: str          
    confidence: str      
    warnings: List[str]

In [11]:
user_profile = {
    "goal": "Improve ML skills",
    "education": "MCA",
    "known_days": 14
}

analytics_summary = {
    "avg_sleep_minutes": 360,
    "avg_work_minutes": 480,
    "avg_exercise_minutes": 20,
    "mean_productivity": 82
}

ml_signals = {
    "productivity_trend": "unstable",
    "habit_risk": "medium"
}

user_feedback = {
    "feels_tired": True
}

In [12]:
def check_profile(profile: Dict) -> List[str]:
    warnings = []

    if not profile.get("goal"):
        warnings.append("User goal not defined")

    if profile.get("known_days", 0) < 7:
        warnings.append("Insufficient historical data")

    return warnings

In [13]:
def detect_risk(analytics: Dict, ml: Dict) -> List[str]:
    warnings = []

    if analytics.get("avg_sleep_minutes", 0) < 360:
        warnings.append("Low sleep detected")

    if ml.get("productivity_trend") == "unstable":
        warnings.append("Productivity instability detected")

    return warnings

In [14]:
def detect_conflict(analytics: Dict, feedback: Dict) -> List[str]:
    warnings = []

    if analytics.get("mean_productivity", 0) > 75 and feedback.get("feels_tired"):
        warnings.append(
            "Conflict: productivity high but user reports fatigue"
        )

    return warnings

In [15]:
def supervisor_decide(
    profile: Dict,
    analytics: Dict,
    ml: Dict,
    feedback: Dict
) -> SupervisorDecision:

    warnings = []

    warnings.extend(check_profile(profile))
    warnings.extend(detect_risk(analytics, ml))
    warnings.extend(detect_conflict(analytics, feedback))

    # Hard block condition
    if "Insufficient historical data" in warnings:
        return SupervisorDecision(
            status="BLOCK",
            confidence="LOW",
            warnings=warnings
        )

    # Confidence assignment
    if len(warnings) >= 2:
        confidence = "LOW"
    elif len(warnings) == 1:
        confidence = "MEDIUM"
    else:
        confidence = "HIGH"

    return SupervisorDecision(
        status="ALLOW",
        confidence=confidence,
        warnings=warnings
    )

In [16]:
decision = supervisor_decide(
    user_profile,
    analytics_summary,
    ml_signals,
    user_feedback
)

decision

SupervisorDecision(status='ALLOW', confidence='LOW', warnings=['Productivity instability detected', 'Conflict: productivity high but user reports fatigue'])

## Supervisor Interpretation

- Supervisor allows LLM reasoning
- Confidence is LOW due to multiple warnings
- LLM must respond cautiously and acknowledge uncertainty